## Домашнее задание по теме “Синтаксический анализ”

### Задание 1: Составление словарей для классификации по тональности

При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.

Составление таких словарей вручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).

Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1)

In [7]:
import json
import pickle
import bz2
import regex
from tqdm import tqdm
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import LabelPropagation

import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline
from nltk.corpus import stopwords
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import nltk
nltk.download('stopwords')
nltk.download('punkt')

Populating the interactive namespace from numpy and matplotlib
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
responses = []
with bz2.BZ2File('/content/drive/MyDrive/HW/NLP/Thematic modeling/Materials/banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

responses[99]

201030it [03:04, 1089.47it/s]


{'author': 'ronnichka',
 'bank_license': 'лицензия № 880',
 'bank_name': 'Югра',
 'city': 'г. Саратов',
 'datetime': '2015-06-03 20:56:57',
 'num_comments': 0,
 'rating_grade': 3,
 'rating_not_checked': False,
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'title': 'Карта ко вкладу'}

In [106]:
df = pd.DataFrame.from_dict(responses).sample(1000)

In [107]:
df.to_pickle('df.zip')
df = pd.read_pickle('df.zip')
df.head(2)

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
109765,г. Москва,False,Опыт общения с банком,1,лицензия № 2289,otem,Русский Стандарт,2012-04-10 16:24:00,Являюсь клиентом Банка уже больше 7 лет. Польз...,5.0
112675,г. Санкт-Петербург,False,Потеряли мои документы,0,лицензия № 1971,hype,Ханты-Мансийский банк Открытие,2012-02-08 14:58:00,Я являюсь индивидуальным предпринимателем. Пыт...,NaN


## 1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.

In [108]:
from nltk.tokenize import sent_tokenize
sentenses = []

for text in tqdm(df['text']):
  for sent in sent_tokenize(text):
    sentenses.append(sent)
print(len(sentenses))
print(sentenses[:3])
print(sentenses[0])

100%|██████████| 1000/1000 [00:00<00:00, 1266.94it/s]

17775
['Являюсь клиентом Банка уже больше 7 лет.', 'Пользуюсь практически всеми продуктами Банка.', 'Никаких проблем не возникало.']
Являюсь клиентом Банка уже больше 7 лет.


In [109]:
m = MorphAnalyzer()
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д','вы']
def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""
mystoplemmas = stopwords.words('russian') + ['который','прошлый','сей','свой','наш','мочь','г','в','и','не','на','что','с','по','т', 'д','для','я']

def lemmatize(text):
    try:
        return " ".join([m.parse(w)[0].normal_form for w in text.split()])  
    except:
        return " "

def preprocessing(text):
    words = words_only(text)
    no_stopwords = remove_stopwords(words)
    lemmas = lemmatize(no_stopwords)
    return(lemmas)

In [110]:
sentenses_lemma = []

for sent in tqdm(sentenses[:]):
    sentenses_lemma.append(preprocessing(sent).split())
print(sentenses_lemma[:10])

 72%|███████▏  | 1927359/2664456 [1:04:18<21:50, 562.27it/s]

In [111]:
len(sentenses)

17775

In [112]:
with open('sentenses_lemma', 'wb') as fp:
    pickle.dump(sentenses_lemma, fp)

In [113]:
with open('sentenses_lemma', 'rb') as f:
    sentenses_lemma = pickle.load(f)

In [114]:
len(sentenses_lemma)

17775

# 2. Обучите по коллекции предложений word2vec

In [115]:
%%time 
model_wv = Word2Vec(sentenses_lemma, size=100, window=3, min_count=5, workers=12,sg=True)

CPU times: user 4.55 s, sys: 38.5 ms, total: 4.59 s
Wall time: 2.82 s


# 3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
-  тест на определение ближайших слов
-  тест на аналогии (мужчина – король : женщина – королева)
-  тест на определение лишнего слова.

тест на определение ближайших слов

In [116]:
model_wv.wv.most_similar('банк')

[('оао', 0.8831881880760193),
 ('россия', 0.8771193623542786),
 ('московский', 0.8755611181259155),
 ('ру', 0.8739587068557739),
 ('отдел', 0.8712854385375977),
 ('филиал', 0.869068443775177),
 ('обслуживаться', 0.8657262325286865),
 ('помощь', 0.8649725317955017),
 ('русский', 0.863423764705658),
 ('рекомендовать', 0.8607878684997559)]

In [117]:
model_wv.wv.most_similar('обслуживание')

[('банковский', 0.9125387072563171),
 ('высокий', 0.899440348148346),
 ('продукт', 0.894034743309021),
 ('пользоваться', 0.8926050066947937),
 ('годовой', 0.8889594078063965),
 ('покупка', 0.8831347227096558),
 ('хороший', 0.8814541101455688),
 ('стандарт', 0.8805955052375793),
 ('условие', 0.8796179294586182),
 ('зарплатный', 0.8789801001548767)]

In [118]:
model_wv.wv.most_similar('документ')

[('копия', 0.875914454460144),
 ('паспорт', 0.870043158531189),
 ('предоставить', 0.8626114726066589),
 ('бумага', 0.8527774810791016),
 ('пакет', 0.8511254787445068),
 ('анкета', 0.8506284952163696),
 ('суд', 0.8412240743637085),
 ('какой', 0.8359270691871643),
 ('трудовой', 0.8326266407966614),
 ('подтверждать', 0.8294145464897156)]

In [119]:
model_wv.wv.most_similar('банкомат')

[('терминал', 0.9119689464569092),
 ('онлайн', 0.8781046867370605),
 ('касса', 0.8711178302764893),
 ('снимать', 0.8655828237533569),
 ('снять', 0.8566663265228271),
 ('система', 0.8538249135017395),
 ('карточный', 0.8472257852554321),
 ('снятие', 0.8472064137458801),
 ('перевод', 0.8470110297203064),
 ('внесение', 0.8468085527420044)]

In [120]:
model_wv.wv.most_similar('кредит')

[('потребительский', 0.8910477161407471),
 ('задолженность', 0.8563479781150818),
 ('досрочно', 0.8480701446533203),
 ('погашение', 0.8462268114089966),
 ('взять', 0.8389360904693604),
 ('погасить', 0.8382972478866577),
 ('досрочный', 0.8382288813591003),
 ('ипотечный', 0.819446325302124),
 ('страховка', 0.8088787198066711),
 ('оформление', 0.8073446750640869)]

In [121]:
model_wv.wv.most_similar('ипотека')

[('дог', 0.995536208152771),
 ('товар', 0.9954859018325806),
 ('кредитование', 0.9945930242538452),
 ('привлекательный', 0.9945592880249023),
 ('авто', 0.994384229183197),
 ('овердрафт', 0.9937148094177246),
 ('уменьшить', 0.9936739206314087),
 ('квартира', 0.9933629631996155),
 ('частичный', 0.992904543876648),
 ('ниже', 0.9928672909736633)]

тест на аналогии (мужчина – король : женщина – королева)

In [122]:
model_wv.most_similar(positive=['рубль',"доллар"], negative=["российский"])

[('размер', 0.9180782437324524),
 ('руб', 0.9075544476509094),
 ('списать', 0.9055261611938477),
 ('комиссия', 0.9030866622924805),
 ('наличный', 0.8971932530403137),
 ('сумма', 0.895738959312439),
 ('внести', 0.8869867324829102),
 ('остаток', 0.8737049102783203),
 ('снятие', 0.8588401675224304),
 ('р', 0.8518273830413818)]

тест на определение лишнего слова

In [123]:
model_wv.doesnt_match("деньги доллар собака".split())

'доллар'

In [124]:
model_wv.doesnt_match("автокредит ипотека потребительский вклад".split())

'вклад'

In [125]:
model_wv.doesnt_match('кабинет комната офис банкомат'.split())

'офис'

## 4. Постройте несколько визуализаций:
- TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
- задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [126]:
top_words = []

from nltk import FreqDist
fd = FreqDist()
for text in sentenses_lemma:
    fd.update(text)
for i in fd.most_common(500):
    top_words.append(i[0])
print(top_words[:15])

['банк', 'карта', 'я', 'деньга', 'день', 'в', 'мой', 'кредит', 'который', 'отделение', 'клиент', 'сотрудник', 'счёт', 'сказать', 'сумма']


In [127]:
len(top_words)

500

In [128]:
top_words = [w for w in top_words if len(w) > 4]
len(top_words)

397

In [129]:
top_words_vec = model_wv[top_words]

In [130]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [131]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="6pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [132]:
plot_1 = model_wv['хороший']-model_wv['плохо']

In [133]:
plot_2 = model_wv['быстро']-model_wv['медленно']

In [134]:
df.head(3)

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
109765,г. Москва,False,Опыт общения с банком,1,лицензия № 2289,otem,Русский Стандарт,2012-04-10 16:24:00,Являюсь клиентом Банка уже больше 7 лет. Польз...,5.0
112675,г. Санкт-Петербург,False,Потеряли мои документы,0,лицензия № 1971,hype,Ханты-Мансийский банк Открытие,2012-02-08 14:58:00,Я являюсь индивидуальным предпринимателем. Пыт...,NaN
80270,г. москва,False,Невозможно дозвониться в ДО!,0,лицензия № 1481,Letto13,Сбербанк России,2013-07-02 18:59:00,Добрый день! Я являюсь клиентом банка по ипоте...,NaN


In [135]:
df.bank_name.value_counts(normalize=1).head(25)

Сбербанк России                   0.164
Альфа-Банк                        0.072
ВТБ 24                            0.065
Русский Стандарт                  0.054
Хоум Кредит Банк                  0.035
Тинькофф Банк                     0.034
Национальный Банк «Траст»         0.030
Московский Кредитный Банк         0.028
Ренессанс Кредит                  0.027
Авангард                          0.025
Связной Банк                      0.023
ОТП Банк                          0.022
Восточный Экспресс Банк           0.021
Ханты-Мансийский банк Открытие    0.017
Райффайзенбанк                    0.017
Ситибанк                          0.016
Промсвязьбанк                     0.014
Банк Москвы                       0.014
ЮниКредит Банк                    0.014
Бинбанк                           0.012
МТС Банк                          0.012
Сетелем Банк                      0.012
Киви Банк                         0.011
БИНБАНК кредитные карты           0.011
Росбанк                           0.010


In [140]:
bank_name=df.bank_name.value_counts(normalize=1).index.tolist()[:12]

In [141]:
bank_name=[preprocessing(bank.lower()).split()[0] for bank in bank_name]
bank_name

['сбербанк',
 'альфа',
 'втб',
 'русский',
 'хоум',
 'тинькофф',
 'национальный',
 'московский',
 'ренессанс',
 'авангард',
 'связный',
 'отп']

In [142]:
df_temp = pd.DataFrame(bank_name,columns=['bank_name'])

In [143]:
df_temp['speed'] = df_temp.apply(lambda  row:np.dot(model_wv[row['bank_name']],plot_1) ,axis=1)

In [144]:
df_temp['quality'] = df_temp.apply(lambda  row:np.dot(model_wv[row['bank_name']],plot_2) ,axis=1)

In [145]:
df_temp.head()

,bank_name,speed,quality
0,сбербанк,0.061354,0.403175
1,альфа,0.281949,0.446888
2,втб,0.214126,0.354832
3,русский,0.407675,0.442775
4,хоум,0.157040,0.390207


In [146]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=df_temp['quality'],
                                    x2=df_temp['speed'],
                                    names=df_temp['bank_name']))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="9pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)
p.xaxis.axis_label = 'Качество'
p.yaxis.axis_label = 'Скорость'
show(p)

### Задание 2:  Распространение метки
Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8 негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:

1. graphlab.label_propagation (graphlab доступен бесплатно по образовательной лицензии)
2. sklearn.semi_supervised.LabelPropagation
3. sklearn.semi_supervised.LabelSpreading

Пример построения графа: см. ноутбук, размещенный в текущей папке репозитория

In [147]:
words = model_wv.wv.vocab.keys()

In [148]:
positive = ['быстрый','удобный','качественный','хороший','красивый' ,'приятный']
negative = ['медленный','долгий','плохой','очередь','ждать' ,'ужасный']

In [149]:
X = positive + negative
y = [1 if x in positive  else 0  if x in negative else -1 for x in words]
y = np.array(y)
y

array([-1, -1, -1, ..., -1, -1, -1])

In [150]:
np.unique(y, return_counts=True)

(array([-1,  0,  1]), array([3246,    6,    6]))

In [151]:
X = model_wv[model_wv.wv.vocab.keys()]

In [152]:
unlabeled_set = np.where(y==-1)

In [153]:
lp_model = LabelSpreading(gamma=1,n_jobs=12)

In [154]:
%%time 
lp_model.fit(X,y)

CPU times: user 436 ms, sys: 201 ms, total: 637 ms
Wall time: 340 ms


LabelSpreading(gamma=1, n_jobs=12)

In [155]:
predicted_labels = lp_model.predict(X[unlabeled_set])
np.unique(predicted_labels, return_counts=True)

(array([0, 1]), array([   3, 3243]))

In [156]:
data = vstack((np.array([*words])[unlabeled_set],predicted_labels))

In [157]:
data = pd.DataFrame({'word':data[0],'label':data[1]})

In [158]:
# Посмотрим на позитивные метки
data[data.label=='1'].head(20)

,word,label
0,являться,1
1,клиент,1
2,банк,1
3,год,1
4,пользоваться,1
5,практически,1
6,весь,1
7,продукт,1
8,никакой,1
9,проблема,1


In [159]:
data[data.label=='0'].head(20)

,word,label
19,день,0
104,горячий,0
105,линия,0
